In [1]:
from Parallelizer import make_parallel
from common import  Common
import re
import pandas as pd
import time
from urllib.parse import unquote
from utils import *
import json
from math import ceil


# TIKTOK

In [2]:
import requests
from bs4 import BeautifulSoup
from time import sleep
count = 0

# @make_parallel
def request_tiktok(product_id):
    payload = {}
    headers = {
        'authority': 'oec-api.tiktokv.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en;q=0.9,en-US;q=0.8',
        'cache-control': 'max-age=0',
        'sec-ch-ua': '"Microsoft Edge";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.47'
    }
    try_time = 0
    json_obj = None
    while not json_obj:
        
        url = f"https://oec-api.tiktokv.com/view/product/{product_id}"
        response = requests.request("GET", url, headers=headers, data=payload)
        try_time += 1
        print(f"{try_time}: {url}")
        try:
            soup = BeautifulSoup(response.text, 'html.parser')

            # get content of script id = RENDER_DATA
            script = soup.find('script', id='RENDER_DATA')
            # get content of script
            script_content = script.contents[0]
            # get data from script
            # print(response.text)
            json_obj = json.loads(unquote(script_content))
            # print(json_obj)
        except Exception as e:
            print(e)
            json_obj = None
            print(f"fail: {product_id} {response.text}")
            sleep(2)
    return json_obj

In [20]:
@make_parallel
def transform(response, results=[]):

    product_transformed = {
        "product_id": "failed",
        "name": "failed",
        "link": "failed",
        "min_price": "failed",
        "max_price": "failed",
        "sold_count": "failed",
        "min_original_price": "failed",
        "max_original_price": "failed",
    }
    
    for i in range(5):
        product_transformed[f"image_{i+1}"] = "failed"

    try:
        product_base = response['2']['initialData']['productInfo']['product_base']
        mapping_product_base = {
            "real_price": ["price", "real_price"],
            "original_price": ["price", "original_price"],
            "images": ["images"],
            "sold_count": ["sold_count"],
            "title": ["title"]
        }
        transformed_product_base = Common.mapping_data(
            product_base, mapping_product_base)
        product_id = response['2']['initialData']['productInfo']['product_id']
        print(product_id)
        name = transformed_product_base['title']
        price_pattern = re.compile(r'(\d+\.?\d*)')
        prices = price_pattern.findall(transformed_product_base['real_price'])
        if len(prices) == 2:
            min_price = int(prices[0].replace('.', ''))
            max_price = int(prices[1].replace('.', ''))
        else:
            min_price = None
            max_price = int(prices[0].replace('.', ''))

        original_price = price_pattern.findall(
            transformed_product_base['original_price'])
        if len(original_price) == 2:
            min_original_price = int(original_price[0].replace('.', ''))
            max_original_price = int(original_price[1].replace('.', ''))
        else:
            min_original_price = None
            if len(original_price) == 0:
                max_original_price = None
            else:
                max_original_price = int(original_price[0].replace('.', ''))

        sold_count = transformed_product_base['sold_count']

        images = [None]*5
        for i,image in enumerate(transformed_product_base["images"][:5]):
            images[i] = f'=image("{image["thumb_url_list"][0]}")'

        product_transformed = {
            "product_id": product_id,
            "name": name,
            "link": f"https://oec-api.tiktokv.com/view/product/{product_id}",
            "min_price": min_price,
            "max_price": max_price,
            "sold_count": sold_count,
            "min_original_price": min_original_price,
            "max_original_price": max_original_price,
        }
        
        for i in range(5):
            product_transformed[f"image_{i+1}"] = images[i]
            
    except Exception as e:
        print(e)
        if product_id:=response['_location'].split('/')[-1]:
            product_transformed["product_id"] = product_id
            product_transformed['link'] =  f"https://oec-api.tiktokv.com/view/product/{product_id}"
            print(f"failed {product_id}")

    results.append(product_transformed)

In [21]:
with open("tiktok_product_info.json", "r", encoding="utf8") as f:
    product_infos = json.load(f)

In [22]:
results = []
transform(product_infos, results)

1729592140551784572
1729449511894157436
1729658387123832956
1729431748019325052
1729626317079218300
1729572679537690748
'price'
'price'
1729440105127118972
expected string or bytes-like object
failed 1729440105127118972
1729451561887434876
1729437324925110396
1729630659136817276
1729583724521031804
'NoneType' object is not subscriptable
failed 1729626846098131068
1729509471228692604
1729605709738772604
1729565467366295676
1729620755435915388
1729446849650919548
1729512000393873532
1729535201399441532
1729599337592555644
1729553523643287676
1729479614583113852
1729501770531702908
1729572678106318972
1729475429215078524
1729601206794225788
1729501769869265020
1729691242946267260
1729431747192129660
1729439080686717052
1729639120843212924
'NoneType' object is not subscriptable
failed 1729626623443372156
1729535202367146108
1729432692253296764
'NoneType' object is not subscriptable
failed 1729624758677244028
1729557070413596796
1729431748429908092
1729431747187804284
1729502787769043068
17

[]

In [23]:
df = pd.DataFrame(results)
df.to_csv("tiktok_product_info.csv", index=False)
df.shape

(190, 13)

# Shopee

In [2]:
from pathlib import Path

In [3]:
all_items = []

In [4]:
for file in Path("atino.vn").glob("*.json"):
    with open(file, "r", encoding="utf8") as f:
        product_infos = json.load(f)
    all_items.extend(product_infos['data']['items'])

In [5]:
with open("sold_out.json", "r", encoding="utf8") as f:
        product_infos = json.load(f)
for product_info in product_infos:
    all_items.extend(product_info['items'])

In [6]:
results = []

In [10]:


def transform_product(item):
    try:
        if "item_basic" in item:
            item = item['item_basic']
        product_id = item['itemid']
        shop_id = item['shopid']
        name = item['name']
        link = transform_link(name,shop_id,product_id)
        max_price = item['price_max'] /100000
        min_price = item['price_min'] / 100000 if item['price_min'] and item['price_min'] !=item['price_max'] else None
        sold_count = int(round(item['historical_sold']/100) * 100) if item['historical_sold'] >=1000 else item['historical_sold']
        max_original_price = item['price_max_before_discount'] / 100000 if item['price_max_before_discount'] > 0 else None
        min_original_price = item['price_min_before_discount'] / 100000 if item['price_min_before_discount'] >0 and item['price_min_before_discount'] != item['price_max_before_discount']  else None
        images = [None]*5
        for i,image in enumerate(item["images"][:5]):
            images[i] = f'=image("https://down-vn.img.susercontent.com/file/{image}")'
            
        product_transformed = {
            "product_id": product_id,
            "name": name,
            "link": link,
            "min_price": min_price,
            "max_price": max_price,
            "sold_count": sold_count,
            "min_original_price": min_original_price,
            "max_original_price": max_original_price,
        }
        
        for i in range(5):
            product_transformed[f"image_{i+1}"] = images[i]
        return product_transformed
    except Exception as e:
        print(e)
        print(json.dumps(item, indent=4))
    

In [11]:
transformed_products = []
for item in all_items:
    transformed_products.append(transform_product(item))

In [12]:
df = pd.DataFrame(transformed_products)
df.to_csv("atino_product_info.csv", index=False)

In [19]:
round(1851/100)*100

1900

In [15]:
for item in all_items:
    if item['itemid'] == 10920498188:
        print(json.dumps(item))

{"itemid": 10920498188, "shopid": 439115986, "name": "\u00c1o Polo Nam NELI ATINO vi\u1ec1n c\u1ed5 thanh l\u1ecbch v\u1ea3i c\u00e1 s\u1ea5u Cotton cao c\u1ea5p tho\u00e1ng m\u00e1t chu\u1ea9n form PL4.3036", "label_ids": [1000268, 1000279, 1008793, 1000277, 1009763, 1006856, 1012191, 1014184, 1014267, 1000260, 1000264, 700025213, 1400085211, 1133882, 1115185, 700066642, 700085482, 1400066702, 1400095073, 700085579, 1400085528, 1213576, 1000269, 1213632, 1243613, 1400085529, 700150015, 700155008, 1208670, 1278591, 1278592, 1268596, 1268599, 1263605, 1278629, 700185064, 700190078, 700195073, 700190079, 1400170039, 1400175056, 1400170062, 1278696, 1263701, 1268699, 1428581, 1428593, 1433638, 1428679, 1428689, 1428733, 1400290091, 1428749, 1518589, 1518590, 1428713, 298418372, 1583596, 1001044, 700085289, 1115060, 1400095029, 1143714, 1173579, 1243627, 700155004, 700000283, 700000280, 844931086908638, 825249635454464, 1400290069, 1400285090, 1428751, 1433772, 1553576, 298408319, 1588624,